In [1]:
import sys
sys.path.append('..')

from deeptables.models import deeptable,deepnets
from deeptables.utils import consts,dt_logging,batch_trainer

from datasets import utils as dsutils
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve
import pandas as pd
import numpy as np
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.regularizers import l2
from tensorflow import keras
import warnings
warnings.filterwarnings("ignore")

import logging

logging.basicConfig(level=logging.ERROR)
pd.set_option('display.max_rows',500)

/Users/jack/opt/anaconda3/envs/tf_2_0/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
data = dsutils.load_adult()

Base dir:/Users/jack/workspace/aps/deeptables/examples/datasets


In [3]:
conf1 = deeptable.ModelConfig(
    name='conf1',
    apply_gbm_features=True,
    gbm_feature_type='dense',
    auto_discrete=True,
)

conf2 = deeptable.ModelConfig(
    name='conf2',
    fixed_embedding_dim=False,
    embeddings_output_dim=0,
    apply_gbm_features=False, #*
    gbm_feature_type='dense',
    auto_discrete=False, #*

)
bt = batch_trainer.BatchTrainer(data,'x_14',
                                eval_size=0.2,
                                validation_size=0.2,
                                metrics=['AUC','accuracy','recall','precision','f1'], #auc/recall/precision/f1/mse/mae/msle/rmse/r2
                                verbose=1,
                                dt_epochs=15,
                                dt_config=[conf1,conf2],
                                dt_nets=[['dnn_nets'],['cross_nets']],
                               )
ms = bt.start()

2 class detected, {' >50K', ' <=50K'}, so inferred as a [binary classification] task
Start training DT model.['dnn_nets']
metrics:['AUC', 'accuracy', 'recall', 'precision', 'f1']
Fitting model...
X_train.shape:(26048, 14),y_train.shape:(26048,)
2 class detected, {' >50K', ' <=50K'}, so inferred as a [binary classification] task
Preparing features cost:0.035035133361816406
Imputation cost:0.09763789176940918
Categorical encoding cost:0.12423396110534668
Discretization cost:0.06466507911682129
Extracting gbm features cost:0.6760299205780029
fit_transform cost:1.0187559127807617
Injected a callback [EarlyStopping]. monitor:val_AUC, patience:1, mode:max
>>>>>>>>>>>>>>>>>>>>>> Model Desc <<<<<<<<<<<<<<<<<<<<<<< 
---------------------------------------------------------
inputs:
---------------------------------------------------------
['all_categorical_vars: (14)', 'input_continuous_all: (6)', 'input_continuous_gbm_leaves: (100)']
---------------------------------------------------------
emb

In [4]:
ms.leaderboard()

,model,type,*auc,accuracy,recall,precision,f1
0,conf1 - ['dnn_nets'] - eval,eval,0.912652,0.860126,0.728460,0.692737,0.710150
1,conf1 - ['cross_nets'] - eval,eval,0.907294,0.852910,0.520888,0.780822,0.624902
2,conf2 - ['dnn_nets'] - eval,eval,0.905133,0.855366,0.639034,0.715643,0.675172
3,conf2 - ['cross_nets'] - eval,eval,0.882193,0.830186,0.496736,0.694343,0.579148
